# Altong App - 전문의약품 크롤링

네이버 의약품 백과사전에서 '구분'이 '전문의약품'인 약품들 크롤링하여 데이터 수집하는 코드입니다.

### 일단 새 가상환경 생성 해봅시다

In [4]:
# conda 가상환경 생성
!conda create -n medicine_crawler python=3.10 -y

# 가상환경에 패키지 설치
!conda install -n medicine_crawler pandas -y
!conda run -n medicine_crawler pip install playwright nest_asyncio

# Playwright 브라우저 설치
!conda run -n medicine_crawler playwright install chromium

# 주피터 커널로 등록
!conda install -n medicine_crawler ipykernel -y
!conda run -n medicine_crawler python -m ipykernel install --user --name medicine_crawler --display-name "medicine_crawler"

Retrieving notices: done
Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3/envs/medicine_crawler

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.11.4  |       hca03da5_0         128 KB
    expat-2.7.3                |       h982b769_0         139 KB
    libcxx-20.1.8              |       hd7fd590_1         306 KB
    libzlib-1.3.1              |       h5f15de7_0          47 KB
    ncurses-6.5                |       hee39554_0         886 KB
    openssl-3.0.18             |       h9b4081a_0         3.1 MB
    pip-25.3                   |     pyhc872135_0         1.1 MB
    python-3.10.19             |       hf701271_0        11.5 MB
    readline-8.3               |       h0b18652_0         464 KB
    setuptools-80.9.0          |  py310hca03da5_0 

## 먼저 테스트 코드로 url 20개 
1. 정해둔 개수만큼 약 url을 수집한다.
2. 각 url을 들어가서 '구분' 필드 값이 '전문의약품'일 경우에만 데이터를 수집한다
3. 반복

In [58]:
"""
네이버 의약품 백과사전 전문의약품 크롤링 - 테스트 버전
- URL 30개만 수집해서 테스트
- 실제 크롤링과 동일한 방식으로 작동
"""

import asyncio
from playwright.async_api import async_playwright
import pandas as pd
from datetime import datetime
import json
import time

# 주피터 노트북용 이벤트 루프 설정
import nest_asyncio
nest_asyncio.apply()


async def get_medicine_classification(page, url, max_retries=3):
    """
    약품 상세 페이지에서 '구분' 확인
    전문의약품이면 True, 아니면 False 반환
    """
    for attempt in range(max_retries):
        try:
            await page.goto(url, wait_until='networkidle', timeout=30000)
            await asyncio.sleep(2)  # 페이지 로딩 대기
            
            # 테이블에서 '구분' 찾기
            rows = await page.locator('table.tmp_profile_tb tr').all()
            
            for row in rows:
                th_text = await row.locator('th').inner_text()
                if '구분' in th_text:
                    td_text = await row.locator('td').inner_text()
                    classification = td_text.strip()
                    print(f"    구분: {classification}")
                    return classification == '전문의약품'
            
            print(f"    ⚠️ '구분' 필드를 찾을 수 없음")
            return False
            
        except Exception as e:
            print(f"    ⚠️ 구분 확인 실패 (시도 {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                await asyncio.sleep(3)
            else:
                return False


async def scrape_medicine_detail(page, url, max_retries=3):
    """개별 의약품 상세 정보 크롤링"""
    for attempt in range(max_retries):
        try:
            await page.goto(url, wait_until='networkidle', timeout=30000)
            await asyncio.sleep(2)
            
            medicine_data = {
                'url': url,
                'doc_id': url.split('docId=')[1].split('&')[0] if 'docId=' in url else None
            }
            
            # 약품명
            try:
                title = await page.locator('h2.headword').inner_text()
                medicine_data['약품명'] = title.strip()
            except:
                medicine_data['약품명'] = None
            
            # 이미지 URL 추출 (a 태그의 href에서 imageUrl 파라미터 추출)
            try:
                print(f"      🔍 이미지 URL 추출 시도...")
                # span.img_box 안의 a 태그 찾기 (first는 await 필요 없음)
                img_link = page.locator('span.img_box > a').first
                
                # a 태그가 실제로 존재하는지 확인
                count = await img_link.count()
                if count > 0:
                    print(f"      ✓ a 태그 찾음")
                    href = await img_link.get_attribute('href')
                    print(f"      ✓ href 값: {href}")
                    
                    # href에서 imageUrl 파라미터 추출
                    if href and 'imageUrl=' in href:
                        print(f"      ✓ imageUrl 파라미터 존재")
                        import urllib.parse
                        # URL 파싱
                        parsed = urllib.parse.urlparse(href)
                        params = urllib.parse.parse_qs(parsed.query)
                        print(f"      ✓ 파싱된 파라미터: {list(params.keys())}")
                        
                        # imageUrl 파라미터 디코딩
                        if 'imageUrl' in params:
                            image_url = params['imageUrl'][0]
                            print(f"      ✅ 이미지 URL 추출 성공!")
                            print(f"      📷 URL: {image_url[:100]}...")
                            medicine_data['이미지_URL'] = image_url
                        else:
                            print(f"      ❌ imageUrl 파라미터 없음")
                            medicine_data['이미지_URL'] = None
                    else:
                        print(f"      ❌ href에 imageUrl 파라미터 없음")
                        medicine_data['이미지_URL'] = None
                else:
                    print(f"      ❌ a 태그를 찾을 수 없음")
                    medicine_data['이미지_URL'] = None
            except Exception as e:
                print(f"      ⚠️ 이미지 URL 추출 실패: {e}")
                import traceback
                print(f"      상세 에러: {traceback.format_exc()}")
                medicine_data['이미지_URL'] = None
            
            # 테이블 정보 추출
            try:
                rows = await page.locator('table.tmp_profile_tb tr').all()
                
                for row in rows:
                    th_text = await row.locator('th').inner_text()
                    td_text = await row.locator('td').inner_text()
                    
                    key = th_text.strip()
                    value = td_text.strip()
                    
                    medicine_data[key] = value
                    
            except Exception as e:
                print(f"      ⚠️ 테이블 파싱 오류: {e}")
            
            # 성분정보
            try:
                content1 = await page.locator('#TABLE_OF_CONTENT1').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['성분정보'] = content1.strip()
            except:
                medicine_data['성분정보'] = None
            
            # 효능효과
            try:
                content2 = await page.locator('#TABLE_OF_CONTENT2').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['효능효과'] = content2.strip()
            except:
                medicine_data['효능효과'] = None
            
            # 용법용량
            try:
                content3 = await page.locator('#TABLE_OF_CONTENT3').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['용법용량'] = content3.strip()
            except:
                medicine_data['용법용량'] = None
            
            # 저장방법
            try:
                content4 = await page.locator('#TABLE_OF_CONTENT4').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['저장방법'] = content4.strip()
            except:
                medicine_data['저장방법'] = None
            
            # 사용기간
            try:
                content5 = await page.locator('#TABLE_OF_CONTENT5').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['사용기간'] = content5.strip()
            except:
                medicine_data['사용기간'] = None
            
            # 사용상의주의사항
            try:
                content6 = await page.locator('#TABLE_OF_CONTENT6').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['사용상의주의사항'] = content6.strip()
            except:
                medicine_data['사용상의주의사항'] = None
            
            return medicine_data
            
        except Exception as e:
            print(f"      ⚠️ 크롤링 실패 (시도 {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                await asyncio.sleep(3)
            else:
                return None


async def get_test_urls(page, base_url, max_urls=20):
    """테스트용: 20개의 URL만 수집"""
    all_urls = []
    page_num = 1
    
    print(f"\n{'='*60}")
    print(f"🧪 테스트 모드: 최대 {max_urls}개 URL만 수집")
    print(f"{'='*60}")
    
    while len(all_urls) < max_urls:
        print(f"\n📄 페이지 {page_num} URL 수집 중...")
        
        current_url = f"{base_url}&page={page_num}"
        
        try:
            await page.goto(current_url, wait_until='networkidle', timeout=30000)
            await asyncio.sleep(2)
            
            # 현재 페이지의 모든 약품 링크 수집
            links = await page.locator('ul.content_list li a[href*="entry.naver"]').all()
            
            if not links:
                print(f"✓ 페이지 {page_num}에서 더 이상 링크를 찾을 수 없습니다.")
                break
            
            page_urls = []
            for link in links:
                href = await link.get_attribute('href')
                if href and 'entry.naver' in href:
                    full_url = f"https://terms.naver.com{href}" if href.startswith('/') else href
                    if full_url not in all_urls and full_url not in page_urls:  # 중복 제거
                        page_urls.append(full_url)
                        
                        # max_urls 도달하면 중단
                        if len(all_urls) + len(page_urls) >= max_urls:
                            break
            
            all_urls.extend(page_urls[:max_urls - len(all_urls)])
            print(f"  ✓ {len(page_urls)}개 수집 (누적: {len(all_urls)}개)")
            
            # max_urls 도달하면 종료
            if len(all_urls) >= max_urls:
                print(f"\n✅ 목표 {max_urls}개 URL 수집 완료!")
                break
            
            page_num += 1
            await asyncio.sleep(1.5)
            
        except Exception as e:
            print(f"⚠️ 페이지 {page_num} 처리 중 오류: {e}")
            break
    
    return all_urls


async def main_test():
    """테스트용 메인 크롤링 함수"""
    
    base_url = "https://terms.naver.com/medicineSearch.naver?mode=exteriorSearch&shape=&color=&dosageForm=&divisionLine=&identifier="
    
    professional_medicines = []
    failed_urls = []
    skipped_count = 0
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()
        
        # 1단계: 테스트용 URL 20개만 수집
        print("=" * 60)
        print("🧪 1단계: 테스트용 URL 20개 수집 중...")
        print("=" * 60)
        
        test_urls = await get_test_urls(page, base_url, max_urls=20)
        print(f"\n✅ {len(test_urls)}개의 테스트 URL 수집 완료!\n")
        
        # URL 저장
        with open('test_medicine_urls.json', 'w', encoding='utf-8') as f:
            json.dump(test_urls, f, ensure_ascii=False, indent=2)
        print("✅ 테스트 URL 목록 저장: test_medicine_urls.json\n")
        
        # 2단계: 각 의약품 상세 정보 크롤링 (전문의약품만)
        print("=" * 60)
        print("🧪 2단계: 전문의약품 필터링 및 크롤링 테스트...")
        print("=" * 60)
        
        for idx, url in enumerate(test_urls, 1):
            print(f"\n[{idx}/{len(test_urls)}] 🔍 {url}")
            
            # 먼저 '구분' 확인
            is_professional = await get_medicine_classification(page, url)
            
            if not is_professional:
                skipped_count += 1
                print(f"  ⏭️  전문의약품 아님 - 스킵 (총 스킵: {skipped_count}개)")
                await asyncio.sleep(1)
                continue
            
            # 전문의약품이면 상세 정보 크롤링
            print(f"  ✅ 전문의약품 확인! 상세 정보 수집 중...")
            medicine_data = await scrape_medicine_detail(page, url)
            
            if medicine_data:
                professional_medicines.append(medicine_data)
                print(f"  ✅ 수집 완료 (전문의약품 총 {len(professional_medicines)}개)")
                
                # 데이터 미리보기
                print(f"      📋 약품명: {medicine_data.get('약품명', 'N/A')}")
                print(f"      📋 업체명: {medicine_data.get('업체명', 'N/A')}")
                print(f"      📋 보험코드: {medicine_data.get('보험코드', 'N/A')}")
            else:
                failed_urls.append(url)
                print(f"  ❌ 수집 실패 - 실패 목록에 추가")
            
            await asyncio.sleep(2)
        
        await browser.close()
    
    # 3단계: 테스트 결과 저장
    print("\n" + "=" * 60)
    print("✅ 테스트 완료! 결과 저장 중...")
    print("=" * 60)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 전문의약품 데이터 저장
    if professional_medicines:
        df_test = pd.DataFrame(professional_medicines)
        df_test.to_csv(f'test_professional_medicines_{timestamp}.csv', index=False, encoding='utf-8-sig')
        print(f"\n✅ 테스트 데이터 저장: test_professional_medicines_{timestamp}.csv ({len(professional_medicines)}건)")
    
    # 실패 목록 저장
    if failed_urls:
        with open(f'test_failed_urls_{timestamp}.json', 'w', encoding='utf-8') as f:
            json.dump(failed_urls, f, ensure_ascii=False, indent=2)
        print(f"⚠️  실패한 URL: test_failed_urls_{timestamp}.json ({len(failed_urls)}건)")
    
    # 테스트 통계
    print("\n" + "=" * 60)
    print("📊 테스트 통계")
    print("=" * 60)
    print(f"테스트 URL 수: {len(test_urls)}건")
    print(f"전문의약품: {len(professional_medicines)}건")
    print(f"일반의약품 등 (스킵): {skipped_count}건")
    print(f"크롤링 실패: {len(failed_urls)}건")
    print("=" * 60)
    
    # 결과 미리보기
    if professional_medicines:
        df_result = pd.DataFrame(professional_medicines)
        print("\n📋 수집된 데이터 미리보기:")
        print(df_result[['약품명', '구분', '업체명', '보험코드']].head(10))
        print(f"\n📋 수집된 전체 컬럼 ({len(df_result.columns)}개):")
        print(list(df_result.columns))
    
    return df_test if professional_medicines else None, failed_urls


# 주피터 노트북에서 실행
print("🧪 네이버 의약품 백과사전 크롤링 테스트 시작!\n")
print("⚠️  이 테스트는 20개 URL만 수집하여 작동을 확인합니다.\n")
start_time = time.time()

df_result, failed_list = await main_test()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\n⏱️  총 소요 시간: {elapsed_time/60:.2f}분 ({elapsed_time:.2f}초)")

print("\n✅ 테스트 완료! 문제없으면 전체 크롤링 코드를 실행하세요.")

🧪 네이버 의약품 백과사전 크롤링 테스트 시작!

⚠️  이 테스트는 20개 URL만 수집하여 작동을 확인합니다.

🧪 1단계: 테스트용 URL 20개 수집 중...

🧪 테스트 모드: 최대 20개 URL만 수집

📄 페이지 1 URL 수집 중...
  ✓ 15개 수집 (누적: 15개)

📄 페이지 2 URL 수집 중...
  ✓ 5개 수집 (누적: 20개)

✅ 목표 20개 URL 수집 완료!

✅ 20개의 테스트 URL 수집 완료!

✅ 테스트 URL 목록 저장: test_medicine_urls.json

🧪 2단계: 전문의약품 필터링 및 크롤링 테스트...

[1/20] 🔍 https://terms.naver.com/entry.naver?docId=2146611&cid=51000&categoryId=51000
    구분: 전문의약품
  ✅ 전문의약품 확인! 상세 정보 수집 중...
      🔍 이미지 URL 추출 시도...
      ✓ a 태그 찾음
      ✓ href 값: /imageDetail.naver?docId=2146611&imageUrl=https%3A%2F%2Fdbscthumb-phinf.pstatic.net%2F3323_000_35%2F20231114085111242_AL4ED2AS6.jpg%2F200704590.jpg%3Ftype%3Dm4500_4500_fst_n%26wm%3DY
      ✓ imageUrl 파라미터 존재
      ✓ 파싱된 파라미터: ['docId', 'imageUrl']
      ✅ 이미지 URL 추출 성공!
      📷 URL: https://dbscthumb-phinf.pstatic.net/3323_000_35/20231114085111242_AL4ED2AS6.jpg/200704590.jpg?type=m...
  ✅ 수집 완료 (전문의약품 총 1개)
      📋 약품명: 펠루비정(펠루비프로펜)
      📋 업체명: 대원제약(주)
      📋 보험코드: 671803380

[2/20] 🔍 htt

In [60]:
df_test30 = pd.read_csv("test_professional_medicines_20251129_173755.csv")

In [62]:
df_test30.head()

,url,doc_id,약품명,이미지_URL,분류,구분,업체명,보험코드,성상,제형,...,색깔,크기,식별표기,성분정보,효능효과,용법용량,저장방법,사용기간,사용상의주의사항,분할선
0,https://terms.naver.com/entry.naver?docId=2146...,2146611,펠루비정(펠루비프로펜),https://dbscthumb-phinf.pstatic.net/3323_000_3...,[01140]해열.진통.소염제,전문의약품,대원제약(주),671803380,담황색 원형정제,나정,...,노랑,"(장축)7, (단축)7, (두께)2.2","마크, PE",이 약 1정(110mg) 중\n펠루비프로펜 30.0mg,"1. 다음 질환의 증상이나 징후의 완화 : 골관절염, 류마티스관절염, 요통(허리통증...","성인 : 1회 1정(펠루비프로펜으로서 30mg), 1일 3회 식후 경구 투여한다.","차광기밀용기, 실온보관(1-30℃)",제조일로부터 36 개월,1. 경고\n(1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 ...,NaN
1,https://terms.naver.com/entry.naver?docId=2134...,2134728,슈다페드정(슈도에페드린염산염),https://dbscthumb-phinf.pstatic.net/3323_000_4...,[02290]기타의 호흡기관용약,전문의약품,삼일제약(주),643900710,흰색의 원형 정제,나정,...,하양,"(장축)8.5, (단축)8.5, (두께)3","마크분할선SD분할선마크분할선SD, 마크분할선SD분할선마크분할선SD",1정(235mg) 중\n슈도에페드린염산염 60mg,"다음 질환에 의한 비충혈 완화 : 감기, 부비동염, 상기도 알레르기",○ 성인 및 12세 이상 : 수도에페드린염산염으로서 1회 30 ～ 60 mg 1일 ...,"기밀용기 또는 밀폐용기, 실온(1~30℃)보관, 제조일로부터 36개월",제조일로부터 36 개월,1. 경고\n슈도에페드린 함유 의약품 복용시 급성 전신성 발진성 농포증(AGEP)과...,"+, +"
2,https://terms.naver.com/entry.naver?docId=2141...,2141123,인데놀정10mg(프로프라놀롤염산염),https://dbscthumb-phinf.pstatic.net/3323_000_3...,[02120]부정맥용제,전문의약품,동광제약(주),645902470,백색의 정제,나정,...,하양,"(장축)7.0, (단축)7.0, (두께)2.0","DK, 10",1정 중 100mg\n프로프라놀롤염산염 10mg,"1. 기외수축(상실성, 심실성), 발작성빈맥의 예방, 빈맥성심방세동, 발작성심방세동...","1. 기외수축(상실성, 심실성), 발작성빈맥의 예방, 빈맥성심방세동, 발작성심방세동...",차광밀폐용기,제조일로부터 60 개월,1. 다음 환자에는 투여하지 말 것.\n1) 이 약에 과민반응 환자\n2) 만성 폐...,NaN
3,https://terms.naver.com/entry.naver?docId=2133...,2133054,바난정(세프포독심프록세틸),https://dbscthumb-phinf.pstatic.net/3323_000_3...,"[06180]주로 그람양성, 음성균에 작용하는 것",전문의약품,에이치케이이노엔(주),640000480,흰색-연한노란색의 원형 필름코팅정,필름코팅정,...,하양,"(장축)8.7, (단축)8.7, (두께)4.5",BNT,이 약 1정(230mg) 중\n세프포독심프록세틸 100mg,"(정제)\n○ 유효균종\n포도구균, 연쇄구균, 폐렴연쇄구균, 임균, 펩토연쇄구균, ...",(정제)\n○ 성인 : 세프포독심으로서 1회 100mg(역가)을 1일 2회 식후 경...,"기밀용기, 실온보존",제조일로부터 36 개월,1. 다음 환자에는 투여하지 말 것.\n이 약에 의한 쇽의 병력이 있는 환자\n\n...,NaN
4,https://terms.naver.com/entry.naver?docId=2140...,2140794,코데날정,https://dbscthumb-phinf.pstatic.net/3323_000_3...,[02220]진해거담제,전문의약품,삼아제약(주),645701150,(내수용) 흰색의 원형 정제(수출용) 미황색의 원형 정제,나정,...,하양,"(장축)10.1, (단축)10.1, (두께)3.4","마크, 분할선","1정 (349.47mg) 중-[내수용]\n디히드로코데인타르타르산염 5.0mg, 구아...","기침, 가래",1. 성인 및 15세 이상 청소년 : 1회 2정씩 1일 3회 식후 경구투여한다.\n...,"기밀용기, 실온(1∼30℃)보관",제조일로부터 36 개월,"1. 경고\n중증의 호흡 억제 위험이 증가할 수 있으니 18세 미만의 비만, 폐색성...",-


In [64]:
df_test30.columns

Index(['url', 'doc_id', '약품명', '이미지_URL', '분류', '구분', '업체명', '보험코드', '성상',
       '제형', '모양', '색깔', '크기', '식별표기', '성분정보', '효능효과', '용법용량', '저장방법', '사용기간',
       '사용상의주의사항', '분할선'],
      dtype='object')

In [66]:
df_test30["사용기간"]

0                               제조일로부터 36 개월
1                               제조일로부터 36 개월
2                               제조일로부터 60 개월
3                               제조일로부터 36 개월
4                               제조일로부터 36 개월
5                               제조일로부터 24 개월
6                               제조일로부터 36 개월
7                               제조일로부터 36 개월
8                               제조일로부터 24 개월
9                               제조일로부터 36 개월
10                              제조일로부터 36 개월
11                              제조일로부터 36 개월
12                              제조일로부터 24 개월
13                              제조일로부터 60 개월
14    제조일로부터 36 개월,제조일로부터 36 개월,제조일로부터 24 개월
15                              제조일로부터 36 개월
16                              제조일로부터 48 개월
17                              제조일로부터 36 개월
Name: 사용기간, dtype: object

In [68]:
df_test30.iloc[14]

url         https://terms.naver.com/entry.naver?docId=2144...
doc_id                                                2144900
약품명                                                     고덱스캡슐
이미지_URL     https://dbscthumb-phinf.pstatic.net/3323_000_3...
분류                                              [03910]간장질환용제
구분                                                      전문의약품
업체명                                                 (주)셀트리온제약
보험코드                                                693900080
성상                          황갈색의 분말이 들어있는 상 ·하 적갈색 불투명의 경질캡슐제
제형                                                  경질캡슐제, 산제
모양                                                        장방형
색깔                                                     갈색, 갈색
크기                              (장축)19.10, (단축)6.63, (두께)6.91
식별표기                                                  GODEX마크
성분정보        1캡슐(412mg) 중\n오로트산카르니틴 150mg, 항독성간장엑스 12.5mg, ...
효능효과                                   트란스아미나제(SGPT)가 상승된 간질환
용법용량    

## 진짜 코드 !!

```
📋 주요 기능
1. ✅ 이미지 URL 수정

테스트 코드와 동일한 방식으로 <a> 태그의 imageUrl 파라미터에서 추출

2. ✅ data 폴더에 저장

data/ 폴더 자동 생성
모든 CSV 파일은 data/ 안에 저장

3. ✅ 개수별 파일명

data/200개_01.csv
data/200개_02.csv
data/147개_05.csv (마지막 배치가 147개인 경우)

4. ✅ 중단/재개 기능
저장되는 파일들:

all_medicine_urls.json - 전체 URL 목록 (처음 한 번만 수집)
crawling_progress.json - 진행 상황 (매번 업데이트)

재개 방법:

크롤링 중 중단됨
다시 실행하면 "이어서 하시겠습니까?" 물어봄
y 입력하면 중단된 지점부터 재개
n 입력하면 처음부터 새로 시작
```

In [77]:
"""
네이버 의약품 백과사전 전문의약품 크롤링 - 전체 버전
- 전체 의약품 중 전문의약품만 크롤링
- 중단/재개 기능 포함
- 200개씩 data 폴더에 저장
"""

import asyncio
from playwright.async_api import async_playwright
import pandas as pd
from datetime import datetime
import json
import time
import os
import urllib.parse

# 주피터 노트북용 이벤트 루프 설정
import nest_asyncio
nest_asyncio.apply()

# data 폴더 생성
os.makedirs('data', exist_ok=True)

# 진행 상황 파일 경로
PROGRESS_FILE = 'crawling_progress.json'
ALL_URLS_FILE = 'all_medicine_urls.json'


def save_progress(current_index, total_processed, total_professional):
    """크롤링 진행 상황 저장"""
    progress = {
        'current_index': current_index,
        'total_processed': total_processed,
        'total_professional': total_professional,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    with open(PROGRESS_FILE, 'w', encoding='utf-8') as f:
        json.dump(progress, f, ensure_ascii=False, indent=2)


def load_progress():
    """저장된 진행 상황 불러오기"""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r', encoding='utf-8') as f:
            return json.load(f)
    return None


def save_batch(medicines, batch_number):
    """200개씩 배치 저장"""
    count = len(medicines)
    filename = f'data/{count}개_{batch_number:02d}.csv'
    
    df = pd.DataFrame(medicines)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n💾 배치 저장: {filename} ({count}건)")
    return filename


async def get_medicine_classification(page, url, max_retries=3):
    """약품 상세 페이지에서 '구분' 확인 (전문의약품이면 True)"""
    for attempt in range(max_retries):
        try:
            await page.goto(url, wait_until='networkidle', timeout=30000)
            await asyncio.sleep(2)
            
            # 테이블에서 '구분' 찾기
            rows = await page.locator('table.tmp_profile_tb tr').all()
            
            for row in rows:
                th_text = await row.locator('th').inner_text()
                if '구분' in th_text:
                    td_text = await row.locator('td').inner_text()
                    classification = td_text.strip()
                    return classification == '전문의약품'
            
            return False
            
        except Exception as e:
            if attempt < max_retries - 1:
                await asyncio.sleep(3)
            else:
                return False


async def scrape_medicine_detail(page, url, max_retries=3):
    """개별 의약품 상세 정보 크롤링"""
    for attempt in range(max_retries):
        try:
            await page.goto(url, wait_until='networkidle', timeout=30000)
            await asyncio.sleep(2)
            
            medicine_data = {
                'url': url,
                'doc_id': url.split('docId=')[1].split('&')[0] if 'docId=' in url else None
            }
            
            # 약품명
            try:
                title = await page.locator('h2.headword').inner_text()
                medicine_data['약품명'] = title.strip()
            except:
                medicine_data['약품명'] = None
            
            # 이미지 URL 추출 (a 태그의 href에서 imageUrl 파라미터 추출)
            try:
                img_link = page.locator('span.img_box > a').first
                count = await img_link.count()
                
                if count > 0:
                    href = await img_link.get_attribute('href')
                    
                    if href and 'imageUrl=' in href:
                        parsed = urllib.parse.urlparse(href)
                        params = urllib.parse.parse_qs(parsed.query)
                        
                        if 'imageUrl' in params:
                            medicine_data['이미지_URL'] = params['imageUrl'][0]
                        else:
                            medicine_data['이미지_URL'] = None
                    else:
                        medicine_data['이미지_URL'] = None
                else:
                    medicine_data['이미지_URL'] = None
            except:
                medicine_data['이미지_URL'] = None
            
            # 테이블 정보 추출
            try:
                rows = await page.locator('table.tmp_profile_tb tr').all()
                
                for row in rows:
                    th_text = await row.locator('th').inner_text()
                    td_text = await row.locator('td').inner_text()
                    medicine_data[th_text.strip()] = td_text.strip()
            except:
                pass
            
            # 성분정보
            try:
                content1 = await page.locator('#TABLE_OF_CONTENT1').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['성분정보'] = content1.strip()
            except:
                medicine_data['성분정보'] = None
            
            # 효능효과
            try:
                content2 = await page.locator('#TABLE_OF_CONTENT2').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['효능효과'] = content2.strip()
            except:
                medicine_data['효능효과'] = None
            
            # 용법용량
            try:
                content3 = await page.locator('#TABLE_OF_CONTENT3').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['용법용량'] = content3.strip()
            except:
                medicine_data['용법용량'] = None
            
            # 저장방법
            try:
                content4 = await page.locator('#TABLE_OF_CONTENT4').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['저장방법'] = content4.strip()
            except:
                medicine_data['저장방법'] = None
            
            # 사용기간
            try:
                content5 = await page.locator('#TABLE_OF_CONTENT5').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['사용기간'] = content5.strip()
            except:
                medicine_data['사용기간'] = None
            
            # 사용상의주의사항
            try:
                content6 = await page.locator('#TABLE_OF_CONTENT6').locator('xpath=following-sibling::p[@class="txt"]').first.inner_text()
                medicine_data['사용상의주의사항'] = content6.strip()
            except:
                medicine_data['사용상의주의사항'] = None
            
            return medicine_data
            
        except Exception as e:
            if attempt < max_retries - 1:
                await asyncio.sleep(3)
            else:
                return None


async def get_all_medicine_urls(page, base_url):
    """전체 의약품 URL 목록 수집"""
    all_urls = []
    page_num = 1
    
    while True:
        print(f"\n📄 페이지 {page_num} URL 수집 중...")
        
        current_url = f"{base_url}&page={page_num}"
        
        try:
            await page.goto(current_url, wait_until='networkidle', timeout=30000)
            await asyncio.sleep(2)
            
            # 현재 페이지의 모든 약품 링크 수집
            links = await page.locator('ul.content_list li a[href*="entry.naver"]').all()
            
            if not links:
                print(f"✓ 페이지 {page_num}에서 더 이상 링크를 찾을 수 없습니다.")
                break
            
            page_urls = []
            for link in links:
                href = await link.get_attribute('href')
                if href and 'entry.naver' in href:
                    full_url = f"https://terms.naver.com{href}" if href.startswith('/') else href
                    if full_url not in page_urls:
                        page_urls.append(full_url)
            
            all_urls.extend(page_urls)
            print(f"  ✓ {len(page_urls)}개 URL 수집 (누적: {len(all_urls)}개)")
            
            # 다음 페이지 버튼 확인
            try:
                next_button = page.locator('span.next a')
                if await next_button.count() == 0:
                    print("✓ 마지막 페이지 도달!")
                    break
            except:
                print("✓ 마지막 페이지 도달!")
                break
            
            page_num += 1
            await asyncio.sleep(1.5)
            
        except Exception as e:
            print(f"⚠️ 페이지 {page_num} 처리 중 오류: {e}")
            break
    
    return all_urls


async def main():
    """메인 크롤링 함수"""
    
    base_url = "https://terms.naver.com/medicineSearch.naver?mode=exteriorSearch&shape=&color=&dosageForm=&divisionLine=&identifier="
    
    professional_medicines = []
    current_batch = []
    batch_number = 1
    failed_urls = []
    skipped_count = 0
    
    # 진행 상황 확인
    progress = load_progress()
    start_index = 0
    
    if progress:
        print(f"\n🔄 이전 크롤링 진행 상황 발견!")
        print(f"  - 마지막 처리 인덱스: {progress['current_index']}")
        print(f"  - 처리된 총 개수: {progress['total_processed']}")
        print(f"  - 수집된 전문의약품: {progress['total_professional']}")
        print(f"  - 저장 시각: {progress['timestamp']}")
        
        resume = input("\n이어서 크롤링하시겠습니까? (y/n): ")
        if resume.lower() == 'y':
            start_index = progress['current_index'] + 1
            batch_number = (progress['total_professional'] // 200) + 1
            print(f"✅ 인덱스 {start_index}부터 재개합니다.\n")
        else:
            print("✅ 처음부터 새로 시작합니다.\n")
            start_index = 0
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()
        
        # 1단계: URL 수집 또는 불러오기
        if os.path.exists(ALL_URLS_FILE) and start_index > 0:
            print("=" * 60)
            print("📂 저장된 URL 목록 불러오는 중...")
            print("=" * 60)
            
            with open(ALL_URLS_FILE, 'r', encoding='utf-8') as f:
                all_urls = json.load(f)
            print(f"✅ {len(all_urls)}개의 URL 불러오기 완료!\n")
        else:
            print("=" * 60)
            print("🔍 1단계: 전체 의약품 URL 수집 중...")
            print("=" * 60)
            
            all_urls = await get_all_medicine_urls(page, base_url)
            print(f"\n✅ 총 {len(all_urls)}개의 URL 수집 완료!\n")
            
            # URL 저장
            with open(ALL_URLS_FILE, 'w', encoding='utf-8') as f:
                json.dump(all_urls, f, ensure_ascii=False, indent=2)
            print(f"✅ URL 목록 저장: {ALL_URLS_FILE}\n")
        
        # 2단계: 크롤링
        print("=" * 60)
        print("🔍 2단계: 전문의약품 크롤링 시작...")
        print("=" * 60)
        
        total_professional = progress['total_professional'] if progress and start_index > 0 else 0
        
        for idx in range(start_index, len(all_urls)):
            url = all_urls[idx]
            print(f"\n[{idx + 1}/{len(all_urls)}] 🔍 처리 중...")
            
            # 구분 확인
            is_professional = await get_medicine_classification(page, url)
            
            if not is_professional:
                skipped_count += 1
                if (idx + 1) % 10 == 0:
                    print(f"  ⏭️  스킵 (총 스킵: {skipped_count}개)")
                await asyncio.sleep(1)
                
                # 진행 상황 저장
                save_progress(idx, idx + 1 - start_index, total_professional)
                continue
            
            # 전문의약품 크롤링
            medicine_data = await scrape_medicine_detail(page, url)
            
            if medicine_data:
                current_batch.append(medicine_data)
                total_professional += 1
                print(f"  ✅ 전문의약품 수집 완료 (배치: {len(current_batch)}/200, 총: {total_professional}개)")
                
                # 200개마다 저장
                if len(current_batch) >= 200:
                    save_batch(current_batch, batch_number)
                    batch_number += 1
                    current_batch = []
            else:
                failed_urls.append(url)
                print(f"  ❌ 수집 실패")
            
            # 진행 상황 저장 (매번)
            save_progress(idx, idx + 1 - start_index, total_professional)
            
            await asyncio.sleep(2)
        
        # 남은 데이터 저장
        if current_batch:
            save_batch(current_batch, batch_number)
        
        await browser.close()
    
    # 최종 결과
    print("\n" + "=" * 60)
    print("✅ 크롤링 완료!")
    print("=" * 60)
    
    # 실패 목록 저장
    if failed_urls:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        with open(f'failed_urls_{timestamp}.json', 'w', encoding='utf-8') as f:
            json.dump(failed_urls, f, ensure_ascii=False, indent=2)
        print(f"⚠️  실패한 URL: failed_urls_{timestamp}.json ({len(failed_urls)}건)")
    
    # 통계
    print(f"\n전체 약품 수: {len(all_urls)}건")
    print(f"전문의약품: {total_professional}건")
    print(f"일반의약품 등 (스킵): {skipped_count}건")
    print(f"크롤링 실패: {len(failed_urls)}건")
    print(f"저장된 배치 파일: {batch_number}개")
    print("=" * 60)
    
    # 진행 상황 파일 삭제 (완료되었으므로)
    if os.path.exists(PROGRESS_FILE):
        os.remove(PROGRESS_FILE)
        print("✅ 진행 상황 파일 삭제 완료")
    
    return total_professional


# 실행
print("🚀 네이버 의약품 백과사전 전문의약품 크롤링 시작!\n")
start_time = time.time()

total_count = await main()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\n⏱️  총 소요 시간: {elapsed_time/3600:.2f}시간 ({elapsed_time/60:.2f}분)")
print(f"✅ 총 {total_count}개의 전문의약품 수집 완료!")

🚀 네이버 의약품 백과사전 전문의약품 크롤링 시작!


🔄 이전 크롤링 진행 상황 발견!
  - 마지막 처리 인덱스: 28
  - 처리된 총 개수: 29
  - 수집된 전문의약품: 5
  - 저장 시각: 2025-11-30 01:09:23



이어서 크롤링하시겠습니까? (y/n):  n


✅ 처음부터 새로 시작합니다.

🔍 1단계: 전체 의약품 URL 수집 중...

📄 페이지 1 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 15개)

📄 페이지 2 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 30개)

📄 페이지 3 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 45개)

📄 페이지 4 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 60개)

📄 페이지 5 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 75개)

📄 페이지 6 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 90개)

📄 페이지 7 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 105개)

📄 페이지 8 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 120개)

📄 페이지 9 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 135개)

📄 페이지 10 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 150개)

📄 페이지 11 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 165개)

📄 페이지 12 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 180개)

📄 페이지 13 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 195개)

📄 페이지 14 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 210개)

📄 페이지 15 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 225개)

📄 페이지 16 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 240개)

📄 페이지 17 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 255개)

📄 페이지 18 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 270개)

📄 페이지 19 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 285개)

📄 페이지 20 URL 수집 중...
  ✓ 15개 URL 수집 (누적: 300개)

📄 페이지 21 U

# 총 41개의 200개 csv + 1개의 13개 csv 저장 완료 !
- 이제 이 csv들을 하나의 csv로 합치겠습니다

In [5]:
import pandas as pd
import glob

# 1) 패턴에 맞는 모든 CSV 파일 불러오기
file_list = glob.glob("data/200개_*.csv")

# 2) 읽은 파일을 리스트에 저장
dfs = [pd.read_csv(file) for file in file_list]

# 3) 하나로 합치기
df_all = pd.concat(dfs, ignore_index=True)


In [7]:
df_all.head()

,url,doc_id,약품명,이미지_URL,분류,구분,업체명,보험코드,성상,제형,...,크기,식별표기,분할선,성분정보,효능효과,용법용량,저장방법,사용기간,사용상의주의사항,제품명
0,https://terms.naver.com/entry.naver?docId=5661...,5661792,베스티온정(베포타스틴베실산염),https://dbscthumb-phinf.pstatic.net/3323_000_3...,[01410]항히스타민제,전문의약품,에이치엘비제약(주),647303390,흰색의 원형 필름코팅정제,필름코팅정,...,"(장축)7.1, (단축)7.1, (두께)3.0","EY, 분할선",-,1정(125.4mg) 중\n베포타스틴베실산염 10.0mg,"다년성 알레르기성 비염, 만성 두드러기, 피부질환에 수반된 소양증(습진.피부염, 피...","통상, 성인에게는 베포타스틴베실산염으로서 1회 10mg을 1일 2회 경구투여한다. ...","기밀용기, 실온(1~30℃)보관",제조일로부터 36 개월,1. 다음 환자에게는 투여하지 말것\n이 약 성분에 과민증이 있는 환자\n\n2. ...,NaN
1,https://terms.naver.com/entry.naver?docId=4355...,4355082,모베타스프레이(모메타손푸로에이트),NaN,[01320]이비과용제,전문의약품,(주)마더스제약,622803061,NaN,NaN,...,NaN,NaN,NaN,1mL 중\n모메타손푸로에이트 0.5mg,1. 성인 및 2세 이상의 소아\n: 계절 알레르기비염 및 연중비염. 중등도 ～ 중...,1. 알레르기 비염\n분무기를 1회 누를 때마다 모메타손푸로에이트 현탁액 약 100...,"기밀용기, 실온보관(2～25℃)",제조일로부터 24 개월,1. 다음 환자에는 투여하지 말 것.\n1) 이 약 또는 이 약의 구성성분에 대해 ...,NaN
2,https://terms.naver.com/entry.naver?docId=5769...,5769641,프리피펜주(아세트아미노펜),NaN,[01140]해열.진통.소염제,전문의약품,제이더블유신약(주),NaN,NaN,NaN,...,NaN,NaN,NaN,100ml\n아세트아미노펜 1000.0mg,통증이나 고열로 인하여 신속하게 정맥 투여할 필요가 있거나 다른 경로로 투여할 수 ...,"정맥으로 투여하며, 성인, 체중 33kg(약 11세) 이상인 소아에 한함\n\n체중...","밀봉용기, 실온(1-30℃)보관",제조일로부터 24 개월,1. 경고\n1) 투약 오류의 위험: 밀리그램과 밀리리터의 혼동으로 인한 투약 오류...,NaN
3,https://terms.naver.com/entry.naver?docId=3613...,3613086,이뮤알파주(싸이모신알파1),NaN,[04290]기타의 종양치료제,전문의약품,(주)한국비엠아이,NaN,NaN,NaN,...,NaN,NaN,NaN,1바이알 중\n싸이모신알파1 1.6mg,면역기능이 저하된 고령 환자의 인플루엔자 백신접종시의 보조요법,이 약 900ug/㎡(1바이알)을 백신접종 첫 주부터 4주간 주 2회씩 피하 또는 ...,"밀봉용기, 2~8℃, 차광보관",제조일로부터 36 개월,1. 다음 환자에는 투여하지 말 것\n1) 이 약 및 이 약의 성분에 과민증의 병력...,NaN
4,https://terms.naver.com/entry.naver?docId=6691...,6691324,"아토바미브정10/5mg(에제티미브, 아토르바스타틴칼슘삼...",https://dbscthumb-phinf.pstatic.net/3323_000_3...,[02180]동맥경화용제,전문의약품,(주)유한양행,642106070,분홍색의 장방형 필름코팅정,필름코팅정,...,"(장축)11.6, (단축)5.6, (두께)4.6","EA5, YH",NaN,1정312mg-에제티미브층\n에제티미브 10mg\n\n1정312mg-아토르바스타틴층...,원발성 고콜레스테롤혈증\n원발성 고콜레스테롤혈증(이형접합 가족형 및 비가족형) 또는...,이 약은 식사와 관계없이 1일 1회 투여한다.\n이 약을 투여하기 전 또는 투여 중...,"기밀용기, 실온(1~30℃)보관",제조일로부터 24 개월,1. 경고\n아토르바스타틴 및 동일 계열의 다른 약물에서 미오글로빈뇨에 의한 이차적...,"아토바미브정10/5mg(에제티미브, 아토르바스타틴칼슘삼수화물)"


In [9]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8200 entries, 0 to 8199
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   url       8200 non-null   object
 1   doc_id    8200 non-null   int64 
 2   약품명       8200 non-null   object
 3   이미지_URL   4683 non-null   object
 4   분류        8198 non-null   object
 5   구분        8200 non-null   object
 6   업체명       8200 non-null   object
 7   보험코드      6129 non-null   object
 8   성상        4683 non-null   object
 9   제형        4683 non-null   object
 10  모양        4683 non-null   object
 11  색깔        4683 non-null   object
 12  크기        4683 non-null   object
 13  식별표기      4676 non-null   object
 14  분할선       1504 non-null   object
 15  성분정보      8200 non-null   object
 16  효능효과      8199 non-null   object
 17  용법용량      8200 non-null   object
 18  저장방법      8200 non-null   object
 19  사용기간      8200 non-null   object
 20  사용상의주의사항  8198 non-null   object
 21  제품명       1252

In [11]:
df_13 = pd.read_csv("data/13개_42.csv")

In [13]:
df_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   url       13 non-null     object 
 1   doc_id    13 non-null     int64  
 2   약품명       13 non-null     object 
 3   이미지_URL   7 non-null      object 
 4   분류        13 non-null     object 
 5   구분        13 non-null     object 
 6   업체명       13 non-null     object 
 7   보험코드      10 non-null     float64
 8   성상        7 non-null      object 
 9   제형        7 non-null      object 
 10  모양        7 non-null      object 
 11  색깔        7 non-null      object 
 12  크기        7 non-null      object 
 13  식별표기      7 non-null      object 
 14  성분정보      13 non-null     object 
 15  효능효과      13 non-null     object 
 16  용법용량      13 non-null     object 
 17  저장방법      13 non-null     object 
 18  사용기간      13 non-null     object 
 19  사용상의주의사항  13 non-null     object 
 20  분할선       2 non-null      object 


In [15]:
df_total = pd.concat([df_all, df_13], ignore_index=True)

In [17]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8213 entries, 0 to 8212
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   url       8213 non-null   object
 1   doc_id    8213 non-null   int64 
 2   약품명       8213 non-null   object
 3   이미지_URL   4690 non-null   object
 4   분류        8211 non-null   object
 5   구분        8213 non-null   object
 6   업체명       8213 non-null   object
 7   보험코드      6139 non-null   object
 8   성상        4690 non-null   object
 9   제형        4690 non-null   object
 10  모양        4690 non-null   object
 11  색깔        4690 non-null   object
 12  크기        4690 non-null   object
 13  식별표기      4683 non-null   object
 14  분할선       1506 non-null   object
 15  성분정보      8213 non-null   object
 16  효능효과      8212 non-null   object
 17  용법용량      8213 non-null   object
 18  저장방법      8213 non-null   object
 19  사용기간      8213 non-null   object
 20  사용상의주의사항  8211 non-null   object
 21  제품명       1254

In [23]:
# 4) 저장
df_total.to_csv("prescription_drug_8213.csv", index=False)

In [27]:
df_total.head(2)

,url,doc_id,약품명,이미지_URL,분류,구분,업체명,보험코드,성상,제형,...,크기,식별표기,분할선,성분정보,효능효과,용법용량,저장방법,사용기간,사용상의주의사항,제품명
0,https://terms.naver.com/entry.naver?docId=5661...,5661792,베스티온정(베포타스틴베실산염),https://dbscthumb-phinf.pstatic.net/3323_000_3...,[01410]항히스타민제,전문의약품,에이치엘비제약(주),647303390,흰색의 원형 필름코팅정제,필름코팅정,...,"(장축)7.1, (단축)7.1, (두께)3.0","EY, 분할선",-,1정(125.4mg) 중\n베포타스틴베실산염 10.0mg,"다년성 알레르기성 비염, 만성 두드러기, 피부질환에 수반된 소양증(습진.피부염, 피...","통상, 성인에게는 베포타스틴베실산염으로서 1회 10mg을 1일 2회 경구투여한다. ...","기밀용기, 실온(1~30℃)보관",제조일로부터 36 개월,1. 다음 환자에게는 투여하지 말것\n이 약 성분에 과민증이 있는 환자\n\n2. ...,NaN
1,https://terms.naver.com/entry.naver?docId=4355...,4355082,모베타스프레이(모메타손푸로에이트),NaN,[01320]이비과용제,전문의약품,(주)마더스제약,622803061,NaN,NaN,...,NaN,NaN,NaN,1mL 중\n모메타손푸로에이트 0.5mg,1. 성인 및 2세 이상의 소아\n: 계절 알레르기비염 및 연중비염. 중등도 ～ 중...,1. 알레르기 비염\n분무기를 1회 누를 때마다 모메타손푸로에이트 현탁액 약 100...,"기밀용기, 실온보관(2～25℃)",제조일로부터 24 개월,1. 다음 환자에는 투여하지 말 것.\n1) 이 약 또는 이 약의 구성성분에 대해 ...,NaN


In [31]:
df_total = df_total.drop(columns=['url'])


In [33]:
df_total = df_total.rename(columns={'doc_id': '문서ID'})


In [35]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8213 entries, 0 to 8212
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   문서ID      8213 non-null   int64 
 1   약품명       8213 non-null   object
 2   이미지_URL   4690 non-null   object
 3   분류        8211 non-null   object
 4   구분        8213 non-null   object
 5   업체명       8213 non-null   object
 6   보험코드      6139 non-null   object
 7   성상        4690 non-null   object
 8   제형        4690 non-null   object
 9   모양        4690 non-null   object
 10  색깔        4690 non-null   object
 11  크기        4690 non-null   object
 12  식별표기      4683 non-null   object
 13  분할선       1506 non-null   object
 14  성분정보      8213 non-null   object
 15  효능효과      8212 non-null   object
 16  용법용량      8213 non-null   object
 17  저장방법      8213 non-null   object
 18  사용기간      8213 non-null   object
 19  사용상의주의사항  8211 non-null   object
 20  제품명       1254 non-null   object
dtypes: int64(1), o

In [43]:
df_total.head(3)

,문서ID,약품명,이미지_URL,분류,구분,업체명,보험코드,성상,제형,모양,색깔,크기,식별표기,분할선,성분정보,효능효과,용법용량,저장방법,사용기간,사용상의주의사항
0,5661792,베스티온정(베포타스틴베실산염),https://dbscthumb-phinf.pstatic.net/3323_000_3...,[01410]항히스타민제,전문의약품,에이치엘비제약(주),647303390,흰색의 원형 필름코팅정제,필름코팅정,원형,하양,"(장축)7.1, (단축)7.1, (두께)3.0","EY, 분할선",-,1정(125.4mg) 중\n베포타스틴베실산염 10.0mg,"다년성 알레르기성 비염, 만성 두드러기, 피부질환에 수반된 소양증(습진.피부염, 피...","통상, 성인에게는 베포타스틴베실산염으로서 1회 10mg을 1일 2회 경구투여한다. ...","기밀용기, 실온(1~30℃)보관",제조일로부터 36 개월,1. 다음 환자에게는 투여하지 말것\n이 약 성분에 과민증이 있는 환자\n\n2. ...
1,4355082,모베타스프레이(모메타손푸로에이트),NaN,[01320]이비과용제,전문의약품,(주)마더스제약,622803061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1mL 중\n모메타손푸로에이트 0.5mg,1. 성인 및 2세 이상의 소아\n: 계절 알레르기비염 및 연중비염. 중등도 ～ 중...,1. 알레르기 비염\n분무기를 1회 누를 때마다 모메타손푸로에이트 현탁액 약 100...,"기밀용기, 실온보관(2～25℃)",제조일로부터 24 개월,1. 다음 환자에는 투여하지 말 것.\n1) 이 약 또는 이 약의 구성성분에 대해 ...
2,5769641,프리피펜주(아세트아미노펜),NaN,[01140]해열.진통.소염제,전문의약품,제이더블유신약(주),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100ml\n아세트아미노펜 1000.0mg,통증이나 고열로 인하여 신속하게 정맥 투여할 필요가 있거나 다른 경로로 투여할 수 ...,"정맥으로 투여하며, 성인, 체중 33kg(약 11세) 이상인 소아에 한함\n\n체중...","밀봉용기, 실온(1-30℃)보관",제조일로부터 24 개월,1. 경고\n1) 투약 오류의 위험: 밀리그램과 밀리리터의 혼동으로 인한 투약 오류...


In [39]:
df_total = df_total.drop(columns=['제품명'])

In [41]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8213 entries, 0 to 8212
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   문서ID      8213 non-null   int64 
 1   약품명       8213 non-null   object
 2   이미지_URL   4690 non-null   object
 3   분류        8211 non-null   object
 4   구분        8213 non-null   object
 5   업체명       8213 non-null   object
 6   보험코드      6139 non-null   object
 7   성상        4690 non-null   object
 8   제형        4690 non-null   object
 9   모양        4690 non-null   object
 10  색깔        4690 non-null   object
 11  크기        4690 non-null   object
 12  식별표기      4683 non-null   object
 13  분할선       1506 non-null   object
 14  성분정보      8213 non-null   object
 15  효능효과      8212 non-null   object
 16  용법용량      8213 non-null   object
 17  저장방법      8213 non-null   object
 18  사용기간      8213 non-null   object
 19  사용상의주의사항  8211 non-null   object
dtypes: int64(1), object(19)
memory usage: 1.3+ MB


In [45]:
# 분류 컬럼이 비어있는 행 삭제
df_total = df_total.dropna(subset=['분류'])


In [49]:
df_total = df_total.drop(columns=["성상", "제형", "모양", '색깔', '크기', '식별표기', '분할선', '보험코드'])

In [51]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8211 entries, 0 to 8212
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   문서ID      8211 non-null   int64 
 1   약품명       8211 non-null   object
 2   이미지_URL   4690 non-null   object
 3   분류        8211 non-null   object
 4   구분        8211 non-null   object
 5   업체명       8211 non-null   object
 6   성분정보      8211 non-null   object
 7   효능효과      8210 non-null   object
 8   용법용량      8211 non-null   object
 9   저장방법      8211 non-null   object
 10  사용기간      8211 non-null   object
 11  사용상의주의사항  8209 non-null   object
dtypes: int64(1), object(11)
memory usage: 1.1+ MB


In [53]:
# 효늫효과 컬럼이 비어있는 행 삭제
df_total = df_total.dropna(subset=['효능효과'])


In [55]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8210 entries, 0 to 8212
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   문서ID      8210 non-null   int64 
 1   약품명       8210 non-null   object
 2   이미지_URL   4690 non-null   object
 3   분류        8210 non-null   object
 4   구분        8210 non-null   object
 5   업체명       8210 non-null   object
 6   성분정보      8210 non-null   object
 7   효능효과      8210 non-null   object
 8   용법용량      8210 non-null   object
 9   저장방법      8210 non-null   object
 10  사용기간      8210 non-null   object
 11  사용상의주의사항  8208 non-null   object
dtypes: int64(1), object(11)
memory usage: 833.8+ KB


In [57]:
# 사용상의주의사항 컬럼이 비어있는 행 삭제
df_total = df_total.dropna(subset=['사용상의주의사항'])

In [59]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8208 entries, 0 to 8212
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   문서ID      8208 non-null   int64 
 1   약품명       8208 non-null   object
 2   이미지_URL   4688 non-null   object
 3   분류        8208 non-null   object
 4   구분        8208 non-null   object
 5   업체명       8208 non-null   object
 6   성분정보      8208 non-null   object
 7   효능효과      8208 non-null   object
 8   용법용량      8208 non-null   object
 9   저장방법      8208 non-null   object
 10  사용기간      8208 non-null   object
 11  사용상의주의사항  8208 non-null   object
dtypes: int64(1), object(11)
memory usage: 833.6+ KB


In [61]:
df_total = df_total.drop(columns=["구분"])

In [63]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8208 entries, 0 to 8212
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   문서ID      8208 non-null   int64 
 1   약품명       8208 non-null   object
 2   이미지_URL   4688 non-null   object
 3   분류        8208 non-null   object
 4   업체명       8208 non-null   object
 5   성분정보      8208 non-null   object
 6   효능효과      8208 non-null   object
 7   용법용량      8208 non-null   object
 8   저장방법      8208 non-null   object
 9   사용기간      8208 non-null   object
 10  사용상의주의사항  8208 non-null   object
dtypes: int64(1), object(10)
memory usage: 769.5+ KB


In [65]:
df_total.to_csv("prescription_drugs_8208.csv", index=False)

In [71]:
import sys
print(sys.executable)

/opt/anaconda3/envs/medicine_crawler/bin/python


# firebase 에 csv 업로드하기
- 전문의약품(prescription_drugs_8208.csv)
- 일반의약품(oct_drugs_4771.csv)

In [69]:
pip install firebase-admin pandas tqdm # 패키지 설치

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 26.0 MB/s  0:00:00 eta 0:00:01
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached rsa-4.9.1-py3-none-any.whl (34 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
Using cached pyasn1-0.6.1-py3-non